# 시스템 품질 변화로 인한 사용자 불편 예지 AI 경진대회
[주제]
비식별화 된 시스템 기록(로그 및 수치 데이터)을 분석하여 시스템 품질 변화로 사용자에게 불편을 야기하는 요인을 진단

[배경]
다양한 장비/서비스에서 일어나는 시스템 데이터를 통해 사용자의 불편을 예지하기 위해 ‘시스템 데이터’와 ‘사용자 불편 발생 데이터’를 분석하여 불편을 느낀 사용자와 불편 요인들을 찾아주세요.

[목적]
데이터를 통해 사용자가 불편을 느끼는 원인 분석
사용자 관점의 데이터 분석

## EDA

In [1]:
#라이브러리 
import pandas as pd
import numpy as np
#시각화 
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc
sns.set(font="THEGaeiduck",rc={"axes.unicode_minus":False},style="white")

In [8]:
#데이터 불러오기 

##에 발생한 에러로그
#학습 데이터 (user_id : 10000 ~ 24999, 15000명)
tr_err=pd.read_csv('./data/train_err_data.csv') #시스템에 발생한 에러 로그
tr_qual=pd.read_csv('./data/train_quality_data.csv') #시스템 퀄리티 로그
tr_pro=pd.read_csv('./data/train_problem_data.csv') #사용자 불만 및 불만이 접수된 시간
#테스트 데이터(user_id : 30000 ~ 44998, 14999명)
ts_err=pd.read_csv('./data/test_err_data.csv') #시스템에 발생한 에러 로그
ts_qual=pd.read_csv('./data/test_quality_data.csv') #시스템 퀄리티 로그

submission=pd.read_csv('./data/sample_submission.csv') #사용자 불만 확률(0~1) (제출용)

C:\Users\oye01\anaconda3\envs\pydatavenv\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (10,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\oye01\anaconda3\envs\pydatavenv\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (4,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
ts_err

,user_id,time,model_nm,fwver,errtype,errcode
0,30000,20201101030227,model_1,04.16.3553,31,1
1,30000,20201101030227,model_1,04.16.3553,33,2
2,30000,20201101030228,model_1,04.16.3553,15,1
3,30000,20201101030256,model_1,04.16.3553,22,1
4,30000,20201101030300,model_1,04.16.3553,11,1
...,...,...,...,...,...,...
16532643,44998,20201130210050,model_1,04.16.3553,40,0
16532644,44998,20201130211831,model_1,04.16.3553,31,1
16532645,44998,20201130211832,model_1,04.16.3553,15,1
16532646,44998,20201130212259,model_1,04.16.3553,16,1


In [8]:
ts_qual

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
1,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
2,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
3,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
4,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747967,20201125180000,44997,04.22.1750,0.0,0,0.0,0,0,2,0,0,0,0,3,0,0
747968,20201125180000,44997,04.22.1750,0.0,0,0.0,0,0,0,0,0,0,0,3,0,0
747969,20201125180000,44997,04.22.1750,0.0,0,0.0,0,0,0,0,0,0,0,3,0,0
747970,20201125180000,44997,04.22.1750,0.0,0,0.0,0,0,0,0,0,0,0,3,0,0


In [9]:
submission

,user_id,problem
0,30000,0
1,30001,0
2,30002,0
3,30003,0
4,30004,0
...,...,...
14994,44994,0
14995,44995,0
14996,44996,0
14997,44997,0


- 에러 로그 (tr_err)

In [3]:
tr_err.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16554663 entries, 0 to 16554662
Data columns (total 6 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   user_id   int64 
 1   time      int64 
 2   model_nm  object
 3   fwver     object
 4   errtype   int64 
 5   errcode   object
dtypes: int64(3), object(3)
memory usage: 757.8+ MB


In [4]:
tr_err

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0
...,...,...,...,...,...,...
16554658,24999,20201130163051,model_3,05.15.2138,15,1
16554659,24999,20201130172625,model_3,05.15.2138,16,1
16554660,24999,20201130172625,model_3,05.15.2138,4,0
16554661,24999,20201130172631,model_3,05.15.2138,4,0


In [5]:
tr_qual

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
1,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
2,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
3,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
4,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828619,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,1,0,0,0,0,17,0,0
828620,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,0,0,0,0,0,17,0,0
828621,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,3,0,0,0,0,17,0,0
828622,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,0,0,0,0,0,17,0,0


In [6]:
tr_pro

,user_id,time
0,19224,20201102200000
1,23664,20201116140000
2,15166,20201114130000
3,12590,20201108210000
4,15932,20201103210000
...,...,...
5424,20167,20201125120000
5425,16270,20201110120000
5426,19114,20201106230000
5427,21505,20201104110000


In [20]:
ts_err.head()

,user_id,time,model_nm,fwver,errtype,errcode
0,30000,20201101030227,model_1,04.16.3553,31,1
1,30000,20201101030227,model_1,04.16.3553,33,2
2,30000,20201101030228,model_1,04.16.3553,15,1
3,30000,20201101030256,model_1,04.16.3553,22,1
4,30000,20201101030300,model_1,04.16.3553,11,1


In [21]:
ts_qual.head()

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
1,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
2,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
3,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
4,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0


In [ ]:
# time컬럼 datetime으로 변경 ##오류##
# tr_pro['time']=tr_pro.time.apply(pd.to_datetime)
# tr_err['time']=tr_err.time.apply(pd.to_datetime)
# tr_qual['time']=tr_qual.time.apply(pd.to_datetime)
## tr_pro['time']=tr_pro['time'].apply(pd.to_datetime)

In [9]:
# time컬럼 datetime으로 변경
tr_err['time']=pd.to_datetime(tr_err['time'], format='%Y%m%d%H%M%S')
tr_pro['time']=pd.to_datetime(tr_pro['time'], format='%Y%m%d%H%M%S')
tr_qual['time']=pd.to_datetime(tr_qual['time'], format='%Y%m%d%H%M%S')
ts_err['time']=pd.to_datetime(ts_err['time'], format='%Y%m%d%H%M%S')
ts_qual['time']=pd.to_datetime(ts_qual['time'], format='%Y%m%d%H%M%S')

In [ ]:
## datetime으로 바꾸기
train_err['time'] = train_err['time'].apply(lambda x: dt.datetime(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:8]), int(str(x)[8:10])))
train_qual['time'] = train_qual['time'].apply(lambda x: dt.datetime(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:8]), int(str(x)[8:10])))
train_prob['time'] = train_prob['time'].apply(lambda x: dt.datetime(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:8]), int(str(x)[8:10])))

test_err['time'] = test_err['time'].apply(lambda x: dt.datetime(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:8]), int(str(x)[8:10])))
test_qual['time'] = test_qual['time'].apply(lambda x: dt.datetime(int(str(x)[:4]), int(str(x)[4:6]), int(str(x)[6:8]), int(str(x)[8:10])))

In [10]:
tr_qual

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,2020-11-29 09:00:00,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
1,2020-11-29 09:00:00,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
2,2020-11-29 09:00:00,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
3,2020-11-29 09:00:00,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
4,2020-11-29 09:00:00,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828619,2020-11-24 03:30:00,24997,04.22.1778,0.0,0,0.0,0,0,1,0,0,0,0,17,0,0
828620,2020-11-24 03:30:00,24997,04.22.1778,0.0,0,0.0,0,0,0,0,0,0,0,17,0,0
828621,2020-11-24 03:30:00,24997,04.22.1778,0.0,0,0.0,0,0,3,0,0,0,0,17,0,0
828622,2020-11-24 03:30:00,24997,04.22.1778,0.0,0,0.0,0,0,0,0,0,0,0,17,0,0


In [4]:
tr_qual

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,1970-01-01 05:36:41.129090,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
1,1970-01-01 05:36:41.129090,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
2,1970-01-01 05:36:41.129090,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
3,1970-01-01 05:36:41.129090,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
4,1970-01-01 05:36:41.129090,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828619,1970-01-01 05:36:41.124033,24997,04.22.1778,0.0,0,0.0,0,0,1,0,0,0,0,17,0,0
828620,1970-01-01 05:36:41.124033,24997,04.22.1778,0.0,0,0.0,0,0,0,0,0,0,0,17,0,0
828621,1970-01-01 05:36:41.124033,24997,04.22.1778,0.0,0,0.0,0,0,3,0,0,0,0,17,0,0
828622,1970-01-01 05:36:41.124033,24997,04.22.1778,0.0,0,0.0,0,0,0,0,0,0,0,17,0,0


In [11]:
tr_err['fwver'].unique()

array(['05.15.2138', '04.33.1185', '04.33.1261', '04.22.1750',
       '04.22.1778', '04.16.3553', '04.33.1149', '04.16.3571',
       '05.66.3237', '05.66.3571', '03.11.1149', '04.22.1684',
       '03.11.1167', '04.82.1684', '04.82.1778', '04.33.1171',
       '04.73.2237', '10', '04.82.1730', '04.73.2571', '8.5.3',
       '04.22.1666', '03.11.1141', '05.15.2120', '04.33.1125',
       '04.16.3439', '04.22.1442', '04.33.1095', '04.16.3569',
       '05.15.2090', '05.15.3104', '05.15.2122', '04.22.1656',
       '04.16.2641', '05.15.2114', '04.16.3345', '05.15.2092'],
      dtype=object)

In [24]:
tr_err

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0
...,...,...,...,...,...,...
16554658,24999,20201130163051,model_3,05.15.2138,15,1
16554659,24999,20201130172625,model_3,05.15.2138,16,1
16554660,24999,20201130172625,model_3,05.15.2138,4,0
16554661,24999,20201130172631,model_3,05.15.2138,4,0


In [42]:
#데이터 유형
tr_err.info()
#결측치
print(tr_err.isnull())
print(tr_err.isnull().sum()) #sum(axis=0) 누락데이터 개수 구하기

#누락 데이터 꺼내기 *****
print(tr_err[tr_err['errcode'].isnull()])  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16554663 entries, 0 to 16554662
Data columns (total 6 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   user_id   int64 
 1   time      int64 
 2   model_nm  object
 3   fwver     object
 4   errtype   int64 
 5   errcode   object
dtypes: int64(3), object(3)
memory usage: 757.8+ MB
          user_id   time  model_nm  fwver  errtype  errcode
0           False  False     False  False    False    False
1           False  False     False  False    False    False
2           False  False     False  False    False    False
3           False  False     False  False    False    False
4           False  False     False  False    False    False
...           ...    ...       ...    ...      ...      ...
16554658    False  False     False  False    False    False
16554659    False  False     False  False    False    False
16554660    False  False     False  False    False    False
16554661    False  False     False  False    False    Fals

In [45]:
print(len(tr_err))
print(tr_err.count())

print('----')
tr_err['fwver'].value_counts()
print('fmwver:',len(tr_err['fwver'].unique()))
tr_err['user_id'].value_counts()
print('userid:',len(tr_err['user_id'].unique()))
tr_err['model_nm'].value_counts()
print('modelnm:',len(tr_err['model_nm'].unique()))
tr_err['errtype'].value_counts()
print('errtype:',len(tr_err['errtype'].unique()))
tr_err['errcode'].value_counts()
print('errcode:',len(tr_err['errcode'].unique()))

16554663
user_id     16554663
time        16554663
model_nm    16554663
fwver       16554663
errtype     16554663
errcode     16554662
dtype: int64
----
fmwver: 37
userid: 15000
modelnm: 9
errtype: 41
errcode: 2806


In [48]:
# 모델 명과 fmwer 와의 관계  
tr_err.groupby(['model_nm','fwver']).count()

user_id     time  errtype  errcode
model_nm fwver                                         
model_0  04.22.1442     2522     2522     2522     2522
         04.22.1656       39       39       39       39
         04.22.1666        5        5        5        5
         04.22.1684     5554     5554     5554     5554
         04.22.1750  2874213  2874213  2874213  2874213
         04.22.1778  1293946  1293946  1293946  1293946
model_1  04.16.2641      296      296      296      296
         04.16.3345      101      101      101      101
         04.16.3439      142      142      142      142
         04.16.3553  5237816  5237816  5237816  5237816
         04.16.3569      980      980      980      980
         04.16.3571   145156   145156   145156   145156
model_2  04.33.1095      909      909      909      909
         04.33.1125      408      408      408      408
         04.33.1149     3272     3272     3272     3272
         04.33.1171       58       58       58       58
         04.33.1185   963736   963736   963736   963736
         04.33.1261  2504871  2504871  2504871  2504870
model_3  05.15.2090       12       12       12       12
         05.15.2092       18       18       18       18
         05.15.2114        9        9        9        9
         05.15.2120      706      706      706      706
         05.15.2122       23       23       23       23
         05.15.2138  1906479  1906479  1906479  1906479
         05.15.3104      191      191      191      191
model_4  03.11.1141       29       29       29       29
         03.11.1149     2114     2114     2114     2114
         03.11.1167  1505659  1505659  1505659  1505659
model_5  04.82.1684    23554    23554    23554    23554
         04.82.1730       78       78       78       78
         04.82.1778     8601     8601     8601     8601
model_6  10             1534     1534     1534     1534
         8.5.3           212      212      212      212
model_7  05.66.3237    31832    31832    31832    31832
         05.66.3571     3524     3524     3524     3524
model_8  04.73.2237    33537    33537    33537    33537
         04.73.2571     2527     2527     2527     2527

- 모델 nm에 따라 fmwver 다름

In [55]:
#tr_err.groupby(['model_nm','errtype']).count()['model_1'] #왜 안되지?
me=tr_err.groupby(['model_nm','errtype']).count()
me.loc['model_1']


,user_id,time,fwver,errcode
errtype,,,,
1,69,69,69,69
2,655,655,655,655
3,3,3,3,3
4,2720,2720,2720,2720
5,220544,220544,220544,220544
6,8985,8985,8985,8985
7,9807,9807,9807,9807
8,10,10,10,10
9,23,23,23,23


In [56]:
tr_err['errcode'].value_counts()

1                     8906967
0                     2599123
connection timeout    1835600
B-A8002                897863
80                     334018
                       ...   
39613                       1
5995                        1
3464                        1
6410                        1
3264                        1
Name: errcode, Length: 2805, dtype: int64

- 퀄리티 로그 

In [ ]:
tr_qual

In [31]:
tr_qual['user_id'].value_counts()
len(tr_qual['user_id'])

828624

In [57]:
tr_qual.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828624 entries, 0 to 828623
Data columns (total 16 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   time        828624 non-null  datetime64[ns]
 1   user_id     828624 non-null  int64         
 2   fwver       788544 non-null  object        
 3   quality_0   684192 non-null  float64       
 4   quality_1   828624 non-null  int64         
 5   quality_2   788511 non-null  float64       
 6   quality_3   828624 non-null  int64         
 7   quality_4   828624 non-null  int64         
 8   quality_5   828604 non-null  object        
 9   quality_6   828624 non-null  int64         
 10  quality_7   828624 non-null  object        
 11  quality_8   828624 non-null  object        
 12  quality_9   828624 non-null  object        
 13  quality_10  828624 non-null  object        
 14  quality_11  828624 non-null  int64         
 15  quality_12  828624 non-null  int64         
dtypes:

In [59]:
tr_qual.isnull().sum()

time               0
user_id            0
fwver          40080
quality_0     144432
quality_1          0
quality_2      40113
quality_3          0
quality_4          0
quality_5         20
quality_6          0
quality_7          0
quality_8          0
quality_9          0
quality_10         0
quality_11         0
quality_12         0
dtype: int64

In [11]:
qme=tr_qual.groupby(['fwver','time']).count()
## 조건 ##
qme[qme['user_id']>24]


user_id  quality_0  quality_1  quality_2  \
fwver      time                                                            
03.11.1167 2020-11-01 00:40:00       48          0         48         48   
           2020-11-01 00:50:00       48          0         48         48   
           2020-11-01 01:00:00       36          0         36         36   
           2020-11-01 02:30:00       36          0         36         36   
           2020-11-01 02:40:00       48          0         48         48   
...                                 ...        ...        ...        ...   
05.15.2138 2020-11-30 22:30:00       36         36         36         36   
           2020-11-30 22:40:00       48         48         48         48   
           2020-11-30 23:00:00       84         84         84         84   
           2020-11-30 23:10:00       84         84         84         84   
           2020-11-30 23:20:00       84         84         84         84   

                                quality_3  quality_4  quality_5  quality_6  \
fwver      time                                                              
03.11.1167 2020-11-01 00:40:00         48         48         48         48   
           2020-11-01 00:50:00         48         48         48         48   
           2020-11-01 01:00:00         36         36         36         36   
           2020-11-01 02:30:00         36         36         36         36   
           2020-11-01 02:40:00         48         48         48         48   
...                                   ...        ...        ...        ...   
05.15.2138 2020-11-30 22:30:00         36         36         36         36   
           2020-11-30 22:40:00         48         48         48         48   
           2020-11-30 23:00:00         84         84         84         84   
           2020-11-30 23:10:00         84         84         84         84   
           2020-11-30 23:20:00         84         84         84         84   

                                quality_7  quality_8  quality_9  quality_10  \
fwver      time                                                               
03.11.1167 2020-11-01 00:40:00         48         48         48          48   
           2020-11-01 00:50:00         48         48         48          48   
           2020-11-01 01:00:00         36         36         36          36   
           2020-11-01 02:30:00         36         36         36          36   
           2020-11-01 02:40:00         48         48         48          48   
...                                   ...        ...        ...         ...   
05.15.2138 2020-11-30 22:30:00         36         36         36          36   
           2020-11-30 22:40:00         48         48         48          48   
           2020-11-30 23:00:00         84         84         84          84   
           2020-11-30 23:10:00         84         84         84          84   
           2020-11-30 23:20:00         84         84         84          84   

                                quality_11  quality_12  
fwver      time                                         
03.11.1167 2020-11-01 00:40:00          48          48  
           2020-11-01 00:50:00          48          48  
           2020-11-01 01:00:00          36          36  
           2020-11-01 02:30:00          36          36  
           2020-11-01 02:40:00          48          48  
...                                    ...         ...  
05.15.2138 2020-11-30 22:30:00          36          36  
           2020-11-30 22:40:00          48          48  
           2020-11-30 23:00:00          84          84  
           2020-11-30 23:10:00          84          84  
           2020-11-30 23:20:00          84          84  

[10625 rows x 14 columns]

In [14]:
tr_qual[(tr_qual['fwver']=='03.11.1167') & (tr_qual['time']=='2020-11-01 00:40:00')]


,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
178512,2020-11-01 00:40:00,13225,03.11.1167,NaN,0,0.0,0,0,0,0,0,0,0,2,0,0
178513,2020-11-01 00:40:00,13225,03.11.1167,NaN,0,0.0,0,0,0,0,0,0,0,2,0,0
178514,2020-11-01 00:40:00,13225,03.11.1167,NaN,0,0.0,0,0,2,0,0,0,0,2,0,0
178515,2020-11-01 00:40:00,13225,03.11.1167,NaN,0,0.0,0,0,0,0,0,0,0,2,0,0
178516,2020-11-01 00:40:00,13225,03.11.1167,NaN,0,0.0,0,0,0,0,0,0,0,2,0,0
178517,2020-11-01 00:40:00,13225,03.11.1167,NaN,0,0.0,0,0,0,0,0,0,0,2,0,0
178518,2020-11-01 00:40:00,13225,03.11.1167,NaN,0,0.0,0,0,0,0,0,0,0,2,0,0
178519,2020-11-01 00:40:00,13225,03.11.1167,NaN,0,0.0,0,0,0,0,0,0,0,2,0,0
178520,2020-11-01 00:40:00,13225,03.11.1167,NaN,-1,-1.0,0,0,-1,-1,0,0,0,2,-1,0
178521,2020-11-01 00:40:00,13225,03.11.1167,NaN,-1,-1.0,0,0,-1,-1,0,0,0,2,-1,0


In [16]:
tr_qual['quality_0'].value_counts()

 0.0       542790
-1.0       130828
 1.0         2097
 2.0         1252
 3.0          518
            ...  
 957.0          1
 3835.0         1
 959.0          1
 3837.0         1
 2039.0         1
Name: quality_0, Length: 753, dtype: int64

In [17]:
tr_qual['quality_1'].value_counts()

 0      670270
-1      153649
 1        2567
 2        1140
 3         391
 4         202
 5         134
 6          71
 7          50
 8          35
 9          26
 10         24
 11         12
 12          7
 13          6
 14          5
 170         4
 18          4
 20          4
 21          4
 16          3
 17          3
 15          3
 169         2
 23          2
 171         1
 19          1
 35          1
 40          1
 88          1
 117         1
Name: quality_1, dtype: int64

In [18]:
tr_qual['quality_2'].value_counts()

 0.0       632469
-1.0       144392
 1.0         2937
 2.0         1073
 3.0          580
            ...  
 436.0          1
 267.0          1
 4279.0         1
 4289.0         1
 624.0          1
Name: quality_2, Length: 798, dtype: int64

In [19]:
tr_qual['quality_3'].value_counts()

0    828624
Name: quality_3, dtype: int64

In [20]:
tr_qual['quality_4'].value_counts()

0    828624
Name: quality_4, dtype: int64

In [21]:
tr_qual['quality_5'].value_counts()

0         428096
-1        153354
1          56171
2          35978
3          21596
           ...  
8,179          1
1,899          1
3,701          1
4,559          1
33,708         1
Name: quality_5, Length: 4744, dtype: int64

## fwver 비교

In [30]:
model_four_qual=tr_qual[tr_qual['fwver']=='03.11.1167']
display(model_four_qual)

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
4332,2020-11-04 12:30:00,10092,03.11.1167,NaN,0,0.0,0,0,0,0,46,6,0,3,0,0
4333,2020-11-04 12:30:00,10092,03.11.1167,NaN,0,0.0,0,0,0,0,46,6,0,3,0,0
4334,2020-11-04 12:30:00,10092,03.11.1167,NaN,0,0.0,0,0,0,0,46,6,0,3,0,0
4335,2020-11-04 12:30:00,10092,03.11.1167,NaN,4,0.0,0,0,0,46,46,6,0,3,0,0
4336,2020-11-04 12:30:00,10092,03.11.1167,NaN,2,0.0,0,0,0,0,46,6,0,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828511,2020-11-21 11:30:00,24990,03.11.1167,NaN,-1,-1.0,0,0,-1,-1,0,0,0,1,-1,0
828512,2020-11-21 11:30:00,24990,03.11.1167,NaN,-1,-1.0,0,0,-1,-1,0,0,0,1,-1,0
828513,2020-11-21 11:30:00,24990,03.11.1167,NaN,0,0.0,0,0,0,0,0,0,0,1,0,0
828514,2020-11-21 11:30:00,24990,03.11.1167,NaN,0,0.0,0,0,0,0,0,0,0,1,0,0


In [26]:
len(tr_qual[tr_qual['fwver']=='03.11.1167'])

104148

In [39]:
model_four_qual['quality_2'].value_counts()

 0.0       89516
-1.0       13562
 1.0         537
 2.0         111
 3.0          61
           ...  
 162.0         1
 4815.0        1
 124.0         1
 117.0         1
 200.0         1
Name: quality_2, Length: 120, dtype: int64

In [40]:
model_four_qual['user_id'].value_counts()

23065    3828
13280    2544
22902    2448
15485    2268
14856    2112
         ... 
18835      12
20087      12
15198      12
19971      12
10740      12
Name: user_id, Length: 514, dtype: int64

In [36]:
#model_four_qual[].count()

time          104148
user_id       104148
fwver         104148
quality_0          0
quality_1     104148
quality_2     104148
quality_3     104148
quality_4     104148
quality_5     104148
quality_6     104148
quality_7     104148
quality_8     104148
quality_9     104148
quality_10    104148
quality_11    104148
quality_12    104148
dtype: int64

In [31]:
model_four_err=tr_err[tr_err['fwver']=='03.11.1167']
display(model_four_err)

,user_id,time,model_nm,fwver,errtype,errcode
97344,10092,2020-11-02 07:06:54,model_4,03.11.1167,1,0
97345,10092,2020-11-02 07:06:56,model_4,03.11.1167,4,0
97346,10092,2020-11-02 07:07:14,model_4,03.11.1167,11,1
97347,10092,2020-11-02 07:07:14,model_4,03.11.1167,12,1
97348,10092,2020-11-02 07:18:20,model_4,03.11.1167,15,1
...,...,...,...,...,...,...
16551776,24990,2020-11-30 21:10:32,model_4,03.11.1167,4,0
16551777,24990,2020-11-30 21:10:33,model_4,03.11.1167,1,0
16551778,24990,2020-11-30 21:10:51,model_4,03.11.1167,11,1
16551779,24990,2020-11-30 21:10:52,model_4,03.11.1167,12,1


In [27]:
len(tr_err[tr_err['fwver']=='03.11.1167'])

1505659

In [ ]:
#err type 수
tr